In [5]:
import rdkit
from rdkit import Chem
from rdkit.Chem import PandasTools
import pandas as pd

# Load a SMILES file into a pandas DataFrame
def load_smiles_to_dataframe(smiles_file, smiles_col='SMILES', id_col='ID', usecols=[0,1]):
    try:
        # Read only the first two columns (ID and SMILES) to save memory
        df = pd.read_csv(smiles_file, sep='\t', header=0, usecols=usecols)
    except Exception as e:
        print(f"Error reading file {smiles_file}: {e}")
        return None
    # Convert to RDKit DataFrame
    #PandasTools.AddMoleculeColumnToFrame(df, smilesCol=smiles_col, molCol='Molecule', includeFingerprints=False)
    #df = PandasTools.LoadSDF(smiles_file, smilesName=smiles_col, idName=id_col)
    return df

# convert SMILES to canonical form
def convert_smiles_to_canonical(df, smiles_col='SMILES'):
    def to_canonical(smile):
        mol = Chem.MolFromSmiles(smile)
        if mol:
            return Chem.MolToSmiles(mol, canonical=True)
        else:
            return None
    df[smiles_col] = df[smiles_col].apply(to_canonical)
    return df


chemSpace_file = '/data/camd/Database/Pulan_ChemSpace/ChemSpace_Sampled_12_24.smi'
smile_col = 'SMILES'
id_col = 'ID'

cs_df = load_smiles_to_dataframe(chemSpace_file, smiles_col=smile_col, id_col=id_col)
cs_df = convert_smiles_to_canonical(cs_df, smiles_col=smile_col)
print(cs_df.head())

real_file = '/data/camd/Database/Pulan_Enamine/2024.07_Enamine_REAL_DB_9.6M.cxsmiles'
real_df = load_smiles_to_dataframe(real_file, smiles_col=smile_col, id_col=id_col)
# rename real_df columns to SMILES and ID
real_df = real_df.rename(columns={smile_col: 'SMILES', id_col: 'ID'})
real_df = convert_smiles_to_canonical(real_df, smiles_col='SMILES')
print(real_df.head())



# Find duplicates based on SMILES
duplicates = pd.merge(cs_df, real_df, on='SMILES', suffixes=('_chemSpace', '_real'))
print(f"Found {len(duplicates)} duplicates based on SMILES.")
print(duplicates.head())




/tmp/ipykernel_4053100/543131798.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(smiles_file, sep='\t', header=0, usecols=usecols)


TypeError: No registered converter was able to produce a C++ rvalue of type std::__cxx11::basic_string<wchar_t, std::char_traits<wchar_t>, std::allocator<wchar_t> > from this Python object of type float